# Task2 数据读取与数据扩增

采用的方法是定长字符识别

**1.学习内容**

1.python中的图像读取(pillow和opencv)
2.使用pytorch读取数据并实现数据扩增


**2.图像读取**

python图像处理必备两个库:pillow和opencv

2.1 pillow

In [3]:
from PIL import Image,ImageFilter
#PIL中的Image类最常用

#读取图片，图像默认格式RGB
im = Image.open("Lena.jpg")
print(im.format, im.size, im.mode)
#调用xv image viewer 展示图片
im.show()

JPEG (230, 240) RGB


In [9]:
#应用模糊滤镜
im_blur = im.filter(ImageFilter.BLUR)
im_blur.show()
im_blur.save("Lena_blur.jpg")

In [ ]:
#生成缩略图
#thumbnail 和 resize
#thumbnail 会覆盖原图而 resize 会返回一个新的对象
(w, h) = im.size
print(im.size)
im_resize = im.resize((w//2, h//2))
print("this is the size of im_resize: {}".format(im_resize.size))
print("this is the size of im: {}".format(im.size))
im.thumbnail((w//2, h//2))
print("this is the size of im: {}".format(im.size))
im.show()

(230, 240)
this is the size of im_resize: (115, 120)
this is the size of im: (230, 240)
this is the size of im: (115, 120)


2.2 OpenCV

opencv是一个功能非常强大的库，其在计算机视觉以及数字图像处理上有着很深的影响力

In [ ]:
import cv2
#opencv 读取图片，图像通道为BGR
img = cv2.imread('Lena.jpg')
cv2.imwrite("Lena_BRG.jpg", img)

# Opencv默认颜色通道顺序是BRG，转换一下
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.imwrite("Lena_RGB.jpg", img)

#转换为灰度图
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite("Lena_gray.jpg", img)
#Canny边缘检测
edges = cv2.Canny(img, 30, 70)
cv2.imwrite('canny.jpg', edges)


**3.数据扩增（Data Augmentation）**
3.1 什么是数据扩增

数据扩增可以增加训练集的样本，对于深度学习而言非常重要（训练集数据越多越好），可以有效缓解模型的过拟合情况，有效地增强模型的泛化能力

3.2 作用

1.深度学习参数非常之多，而一般数据集很难满足数量上的条件

2.扩展样本空间，比如通过旋转，位移或者颜色变化，让模型更关注需要识别的对象本身，而非对象所在的位置或者角度

3.3数据扩增方法

2.3.2 常见的数据扩增方法
在常见的数据扩增方法中，一般会从图像颜色、尺寸、形态、空间和像素等角度进行变换。当然不同的数据扩增方法可以自由进行组合，得到更加丰富的数据扩增方法。

以torchvision为例，常见的数据扩增方法包括：

+ transforms.CenterCrop 对图片中心进行裁剪 

+ transforms.ColorJitter 对图像颜色的对比度、饱和度和零度进行变换 

+ transforms.FiveCrop 对图像四个角和中心进行裁剪得到五分图像

+ transforms.Grayscale 对图像进行灰度变换

+ transforms.Pad 使用固定值进行像素填充

+ transforms.RandomAffine 随机仿射变换

+ transforms.RandomCrop 随机区域裁剪

+ transforms.RandomHorizontalFlip 随机水平翻转

+ transforms.RandomRotation 随机旋转

+ transforms.RandomVerticalFlip 随机垂直翻转

**2.4 pytorch读取数据**

下面的代码中封装了Dataset和DataLoader两个接口

Dataset：对数据集的封装，提供索引方式的对数据样本进行读取

DataLoder：对Dataset进行封装，提供批量读取的迭代读取

In [ ]:
import os, sys, glob, shutil, json
import cv2

from PIL import Image
import numpy as np

import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms

class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label 
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
        
        # 原始SVHN中类别10为数字0
        lbl = np.array(self.img_label[index], dtype=np.int)
        lbl = list(lbl)  + (5 - len(lbl)) * [10]
        
        return img, torch.from_numpy(np.array(lbl[:5]))

    def __len__(self):
        return len(self.img_path)

train_path = glob.glob('../input/train/*.png')
train_path.sort()
train_json = json.load(open('../input/train.json'))
train_label = [train_json[x]['label'] for x in train_json]

#using Dataset
data = SVHNDataset(train_path, train_label,
          transforms.Compose([
              # 缩放到固定尺寸
              transforms.Resize((64, 128)),

              # 随机颜色变换
              transforms.ColorJitter(0.2, 0.2, 0.2),

              # 加入随机旋转
              transforms.RandomRotation(5),

              # 将图片转换为pytorch 的tesntor
              # transforms.ToTensor(),

              # 对图像像素进行归一化
              # transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]))

#using train_loader
train_loader = torch.utils.data.DataLoader(
        SVHNDataset(train_path, train_label,
                   transforms.Compose([
                       transforms.Resize((64, 128)),
                       transforms.ColorJitter(0.3, 0.3, 0.2),
                       transforms.RandomRotation(5),
                       transforms.ToTensor(),
                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])), 
    batch_size=10, # 每批样本个数
    shuffle=False, # 是否打乱顺序
    num_workers=10, # 读取的线程个数
)

for train_data in train_loader:
    break